# Guliford_model_new

In [197]:
#Import all the necessary libraries
import os 
import time
import re
import numpy as np
import pandas as pd 
                            #For Visualisation 
import matplotlib.pyplot as plt
import seaborn as sns 
                           #For preprocessing

#import nltk
#nltk.download('stopwords')
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords 
from nltk import ConfusionMatrix
#from bs4 import BeautifulSoup
                           #To save model and  load model to re-use
#from keras.models import load_model
#from keras.models import model_from_json

                        #To ignore warning 
import warnings
warnings.filterwarnings("ignore")
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
#from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.grid_search import GridSearchCV 
from sklearn.cross_validation import KFold, train_test_split
from sklearn.metrics import accuracy_score,confusion_matrix,recall_score,precision_recall_curve,auc,roc_curve,roc_auc_score,classification_report
from collections import Counter
             # Sampling techiques 
from imblearn.under_sampling import RandomUnderSampler, NearMiss, CondensedNearestNeighbour
from imblearn.under_sampling import EditedNearestNeighbours, RepeatedEditedNearestNeighbours,TomekLinks
from imblearn.over_sampling import SMOTE, RandomOverSampler
#from kmeans_smote import KMeansSMOTE
from imblearn.combine import SMOTEENN, SMOTETomek
from imblearn.ensemble import BalanceCascade, EasyEnsemble

#from itable import prettyTable, TableStyle, CellStyle
#from imblearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier as KNN

         # MODELS
from sklearn.ensemble import AdaBoostClassifier,RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB, MultinomialNB


In [198]:
os.chdir('E:/VINOD KUMAR/Project_Guf')

In [199]:
#actual_data=pd.read_csv('Guliford_data.csv',header=0)
actual_data=pd.read_csv(open('Guliford_cleaned_data.csv','rU'), encoding='utf-8', engine='c')

In [200]:
test_data=pd.read_csv("Guliford_test_cleaned_data.csv",header=0 ,encoding='utf-8', engine='c')

In [201]:
print(actual_data.shape)
print(test_data.shape)
print(actual_data.columns)
print(test_data.columns)

(86554, 17)
(2279, 17)
Index(['FileID', 'clientid', 'PID', 'SUMMARY', 'DATA', 'Categories1',
       'Sub_categories1', 'Categories2', 'Sub_categories2', 'Categories3',
       'Sub_categories3', 'Categories4', 'Sub_categories4', 'Categories5',
       'Sub_categories5', 'Previous_Appointment', 'pre_data'],
      dtype='object')
Index(['FileID', 'clientid', 'PID', 'SUMMARY', 'DATA', 'Categories1',
       'Sub_categories1', 'Categories2', 'Sub_categories2', 'Categories3',
       'Sub_categories3', 'Categories4', 'Sub_categories4', 'Categories5',
       'Sub_categories5', 'Previous_Appointment', 'pre_data'],
      dtype='object')


In [202]:
actual_data1=actual_data[['SUMMARY', 'pre_data','Categories1','Sub_categories1','Previous_Appointment']]

In [203]:
test_data1=test_data[['SUMMARY', 'pre_data','Categories1','Sub_categories1','Previous_Appointment']]

In [204]:
# Changing the column names
col_names=["summary","data","categories","sub_categories","appointment"]

In [205]:
actual_data1.columns=col_names
test_data1.columns=col_names

In [206]:
del actual_data 
del test_data 

In [207]:
test_data1 = test_data1.replace(np.nan, '', regex=True)
actual_data1 = actual_data1.replace(np.nan, '', regex=True)
#actual_data1.fillna(none, inplace=True)
#actual_data1.replace(np.nan, 0, inplace=True)

In [208]:
#Converting to lower case
actual_data1['summary'] = actual_data1['summary'].astype(str).str.lower()
actual_data1["data"] = actual_data1["data"].astype(str).str.lower()
actual_data1["categories"] = actual_data1["categories"].map(lambda x: x.upper())

In [209]:
#Converting to lower case
test_data1['summary'] = test_data1['summary'].astype(str).str.lower()
test_data1["data"] = test_data1["data"].astype(str).str.lower()
test_data1["categories"] = test_data1["categories"].map(lambda x: x.upper())

In [210]:
print(actual_data1.dtypes)
print(test_data1.dtypes)

summary           object
data              object
categories        object
sub_categories    object
appointment       object
dtype: object
summary           object
data              object
categories        object
sub_categories    object
appointment       object
dtype: object


In [211]:
#Converting datatype of summary feature
actual_data1.summary =actual_data1.summary.astype('str')
#Converting datatype of data feature
actual_data1.data =actual_data1.data.astype('str')
#Converting datatype of category feature
actual_data1.categories=actual_data1.categories.astype("category")

In [212]:
#Converting datatype of summary feature
test_data1.summary =test_data1.summary.astype('str')
#Converting datatype of data feature
test_data1.data =test_data1.data.astype('str')
#Converting datatype of category feature
test_data1.categories=test_data1.categories.astype("category")

In [214]:
print(actual_data1.categories.value_counts())
#Categoreies coulumn levels count 
print(actual_data1.categories.value_counts()/len(actual_data1.data))

ASK_A_DOCTOR     31337
PRESCRIPTION     21490
APPOINTMENTS     19477
MISCELLANEOUS    10848
LAB               3402
Name: categories, dtype: int64
ASK_A_DOCTOR     0.362051
PRESCRIPTION     0.248284
APPOINTMENTS     0.225027
MISCELLANEOUS    0.125332
LAB              0.039305
Name: categories, dtype: float64


In [215]:
test_data1.head()

,summary,data,categories,sub_categories,appointment
0,order for mra,can you reorder as without and i can get it or...,PRESCRIPTION,PROVIDER,No
1,patient sick wants rx,patient called a couple of days ago r t coughi...,ASK_A_DOCTOR,SYMPTOMS,No
2,fyi for dr. russo,patient s wife called and said that her husba...,ASK_A_DOCTOR,SYMPTOMS,No
3,rx refills,ashley cross rn march am nu...,PRESCRIPTION,REFILL,No
4,"frequent falls, refusing care, call from wells...",nursing supervisor at wellspring concerned abo...,ASK_A_DOCTOR,SYMPTOMS,No


In [217]:
#Removing Punctuation
actual_data1['data'] = actual_data1['data'].str.replace('[^\w\s\d\*\\b]','')
actual_data1['summary'] = actual_data1['summary'].str.replace('[^\w\s\d\*\\b]','')  

In [218]:
#Removing Punctuation#Removing Punctuation
actual_data1['data'] = actual_data1['data'].str.replace('[^\w\s\d\*\\b]','')
actual_data1['summary'] = actual_data1['summary'].str.replace('[^\w\s\d\*\\b]','')  
test_data1['data'] = test_data1['data'].str.replace('[^\w\s\d\*\\b]','')
test_data1['summary'] = test_data1['summary'].str.replace('[^\w\s\d\*\\b]','')

In [220]:
#Replacing spelling errors 
import re
def replacing_text(input_text):
    letters_only = re.sub("[^a-zA-Z]", " ",input_text )
    low_case = letters_only.lower()
    string  =re.sub(' pt ', ' patient ', low_case)
    string  =re.sub(' appt ', ' appointment ',  string)
    string  =re.sub(' appts ', ' appointment ',  string)
    string  =re.sub(' ov ', ' office visit ',  string)
    string  =re.sub(' schd ', ' scheduled ',  string)
    string  =re.sub(' np ', ' new patient ',  string)
    string  =re.sub(' r/s ', ' reschedule ',  string)
    string  =re.sub(' med ', ' medication ',  string)
    string  =re.sub(' pa ', ' prior authorization ', string)
    string  =re.sub(' fin ', ' financial ',  string)
    string  =re.sub(' dept ', ' department ',  string) 
    string  =re.sub(' meds ', ' medication ', string)
    string  =re.sub(' tabs ', ' tablets ', string)
    string  =re.sub(' ph ',' phone ', string)
    string  =re.sub(' dr ',' doctor ', string)
    string  =re.sub(' msg ',' message ', string)
    string  =re.sub(' sched ',' scheduled ', string)
    string  =re.sub(' yrly ',' yearly ', string)
    #string  =re.sub(r'xxx ',' ', string)
    string  =re.sub(' clld ', ' called ', string)
    string  =re.sub(' rs ',' reschedule ', string)
    string  =re.sub(' patient ashley ', ' ',  string)
    #string  =re.sub(' rn ', ' relay neurology ', string)
    #string  =re.sub(' ad ', ' add ', string)
    #string  =re.sub(' am ', ' ', string)
    review_text = string
    return(review_text) 

In [221]:
length_data=len(actual_data1)

In [222]:
start = time.time()
data_process2=[]
for i in range (0,length_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    data_process2.append(replacing_text(actual_data1.data[i]))
    
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000
no.of rows reviews processed: 60000
no.of rows reviews processed: 70000
no.of rows reviews processed: 80000
Training took 6.404 seconds


In [223]:
start = time.time()
summary_process2=[]
for i in range (0,length_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    summary_process2.append(replacing_text(actual_data1.summary[i]))
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
no.of rows reviews processed: 10000
no.of rows reviews processed: 20000
no.of rows reviews processed: 30000
no.of rows reviews processed: 40000
no.of rows reviews processed: 50000
no.of rows reviews processed: 60000
no.of rows reviews processed: 70000
no.of rows reviews processed: 80000
Training took 2.612 seconds


In [224]:
actual_data1.data=pd.DataFrame(data_process2)
actual_data1.summary=pd.DataFrame(summary_process2)

In [226]:
length_test_data=len(test_data1)

In [227]:
start = time.time()
test_data_process2=[]
for i in range (0,length_test_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    test_data_process2.append(replacing_text(test_data1.data[i]))
    
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
Training took 0.180 seconds


In [228]:
start = time.time()
test_data_summary_process2=[]
for i in range (0,length_test_data):
    if i%10000 == 0:
        print ("no.of rows reviews processed:",i)
    test_data_summary_process2.append(replacing_text(test_data1.summary[i]))
print("Training took {:0.3f} seconds".format(time.time()-start))

no.of rows reviews processed: 0
Training took 0.068 seconds


In [229]:
test_data1.data=pd.DataFrame(test_data_process2)
test_data1.summary=pd.DataFrame(test_data_summary_process2)

In [230]:
test_data1.head()

,summary,data,categories,sub_categories,appointment
0,order for mra,can you reorder as without and i can get it or...,PRESCRIPTION,PROVIDER,No
1,patient sick wants rx,patient called a couple of days ago r t coughi...,ASK_A_DOCTOR,SYMPTOMS,No
2,fyi for doctor russo,patient s wife called and said that her husba...,ASK_A_DOCTOR,SYMPTOMS,No
3,rx refills,ashley cross rn march am nu...,PRESCRIPTION,REFILL,No
4,frequent falls refusing care call from wellspr...,nursing supervisor at wellspring concerned abo...,ASK_A_DOCTOR,SYMPTOMS,No


In [231]:
#Removal of Stop Words
stop = stopwords.words('english')
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [232]:
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [233]:
actual_data1.data[20]

'bp yesterday saw bp keeps going doctor manny said hr hospital bp normal running high may anxious changing urosotomy bag md new patient visit today check bp willing thank mark perini md january appointment scheduled amanda gray january'

In [234]:
#Removal of user defined Stop Words
stop2 = ['amanda' ,'ashley','gray','john', 'russo', 'md', 'january','pm','am','Lisa','Kirk' ,'ravisankar', 'r','avva', 'mark' , 'perini','phone','patient','please','melissa','dykes','rn','caller','donna','shanon','wright','cma','amy','daughter','taken','take','mg','ann','thank','x','r','call','a','m','mom','ravisankar','home','john','pm','jones','erica','ma','md','called','russo','http','january','february','march','april','may','june','july','august','september','october','november','december']
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))

In [235]:
#Removal of user defined Stop Words
test_data1['data'] = test_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))
test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in stop2))

In [236]:
freq_count_data = pd.Series(' '.join(actual_data1['data']).split()).value_counts()
freq_count_summary = pd.Series(' '.join(actual_data1['summary']).split()).value_counts()
print(freq_count_data,freq_count_summary)

doctor          51524
oral            50548
tablets         38689
rx              37174
one             36948
tablet          35400
daily           34519
said            33965
appointment     32334
notified        30269
day             29282
back            28653
mouth           27881
need            23874
days            23380
new             22400
kelly           22346
get             22167
sykes           21973
know            21368
would           21037
see             20135
pharmacy        19920
po              19814
today           19283
n               18409
needs           18299
pain            18242
medication      17805
like            16798
                ...  
refillss            1
fundraising         1
anxety              1
reliably            1
flucation           1
plexus              1
cephlapine          1
poo                 1
toyed               1
laralee             1
brookwood           1
sjogrens            1
archana             1
ambbien             1
notcied   

In [237]:
freq_data = pd.Series(' '.join(actual_data1['data']).split()).value_counts()[:10]
print(freq_data)
freq_summary = pd.Series(' '.join(actual_data1['summary']).split()).value_counts()[:10]
print(freq_summary)

doctor         51524
oral           50548
tablets        38689
rx             37174
one            36948
tablet         35400
daily          34519
said           33965
appointment    32334
notified       30269
dtype: int64
done           10870
rx              6406
doctor          6086
refill          4801
ask             4178
needs           3896
pain            3780
appointment     3582
appt            3455
wants           3149
dtype: int64


In [238]:
#removing commonly occurring words
freq_data_l = list(freq_data.index)
actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_data_l))
freq_summary_l = list(freq_summary.index)
actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join(x for x in x.split() if x not in freq_summary_l))


In [239]:
print(len(freq_count_data),len(freq_count_summary))
print(len(set(freq_count_data)),len(set(freq_count_summary)))

40487 12931
1347 343


In [269]:
#stemmer = SnowballStemmer('english')
#words = stopwords.words("english")

In [270]:
#actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
#actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())


In [271]:
#test_data1['data'] = test_data1['data'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())
#test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split() if i not in words]).lower())


In [272]:
#test_data1.data[0]

In [273]:
#actual_data1.data[0]

In [146]:
###############################################################################################################################

In [147]:
#pd.DataFrame(freq_count_summary).to_csv('Guliford_data_freq_count_summary.csv')
#pd.DataFrame(freq_count_data).to_csv('Guliford_data_freq_count_data.csv')

In [148]:
#from nltk.stem import PorterStemmer
#from nltk.stem.lancaster import LancasterStemmer
#st = PorterStemmer()
#lt = LancasterStemmer()
#actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [149]:
#test_data1['data'] = test_data1['data'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))
#test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join([st.stem(word) for word in x.split()]))

In [150]:
#actual_data1['data'] = actual_data1['data'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))
#actual_data1['summary'] = actual_data1['summary'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))

In [151]:
#test_data1['data'] = test_data1['data'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))
#test_data1['summary'] = test_data1['summary'].apply(lambda x: " ".join([lt.stem(word) for word in x.split()]))

In [242]:
#doing TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_data_vectorizer = TfidfVectorizer(ngram_range=(1,2),norm="l2",max_features= 3000)
tfidf_data = tfidf_data_vectorizer.fit_transform(actual_data1["data"])
dense_data = tfidf_data.todense()

tfidf_summary_vectorizer = TfidfVectorizer(ngram_range=(1,2),norm="l2",max_features= 1000)
tfidf_summary = tfidf_test_data = tfidf_summary_vectorizer.fit_transform(actual_data1["summary"])
dense_summary= tfidf_summary.todense()

tfidf_test_summary = tfidf_summary_vectorizer.fit_transform(test_data1["summary"])
dense_test_summary = tfidf_test_summary.todense()

tfidf_test_data = tfidf_data_vectorizer.fit_transform(test_data1["data"])
dense_test_data = tfidf_test_data.todense()

df_test_summary = pd.DataFrame(dense_test_summary) 
df_test_data = pd.DataFrame(dense_test_data)

df_data = pd.DataFrame(dense_data)
df_summary = pd.DataFrame(dense_summary)

In [243]:
tfidf_test_data = tfidf_data_vectorizer.fit_transform(test_data1["data"])
dense_test_data = tfidf_test_data.todense()

tfidf_test_summary = tfidf_summary_vectorizer.fit_transform(test_data1["summary"])
dense_test_summary = tfidf_test_summary.todense()

df_test_data = pd.DataFrame(dense_test_data)
df_test_summary = pd.DataFrame(dense_test_summary)

In [244]:
feature_names_data = tfidf_data_vectorizer.get_feature_names()
print(len(feature_names_data))
print(feature_names_data[:3000])

3000
['ab', 'abd', 'abdomen', 'abdominal', 'able', 'able get', 'abnormal', 'abx', 'accu', 'accu chek', 'acetaminophen', 'acetaminophen one', 'acetaminophen oral', 'acetaminophen tablet', 'ache', 'aches', 'acid', 'act', 'act inhalation', 'act nasal', 'acting', 'activated', 'active', 'actually', 'acute', 'add', 'added', 'adderall', 'adderall oral', 'additional', 'address', 'adjuvanted', 'adjuvanted inject', 'administer', 'admitted', 'advanced', 'advice', 'advil', 'advise', 'advised', 'advised come', 'advised kelly', 'advised rx', 'advised would', 'advising', 'aerosol', 'aerosol powder', 'aerosol solution', 'affected', 'affected area', 'affected eye', 'afternoon', 'age', 'agitation', 'ago', 'agree', 'agreeable', 'agreeable appointment', 'agreeable rogers', 'agreeable see', 'agreeable weaver', 'agreed', 'ahc', 'ahead', 'aid', 'air', 'alamance', 'alamance church', 'albuterol', 'alcohol', 'align', 'align one', 'allergic', 'allergic medications', 'allergies', 'allergist', 'allergy', 'allopuri

In [245]:
feature_names_summary = tfidf_summary_vectorizer.get_feature_names()
print(len(feature_names_summary))
print(feature_names_summary[:1000])

1000
['abd', 'abx', 'abx done', 'abx request', 'abx requested', 'abx sinus', 'aches', 'aches cough', 'acid', 'add', 'add labs', 'adderall', 'adderall rx', 'admitted', 'advice', 'agitation', 'ahc', 'ahc check', 'ahc req', 'aid', 'aid done', 'alcohol', 'allergy', 'allopurinol', 'alprazolam', 'alprazolam refill', 'alternative', 'ambien', 'anderson', 'ankle', 'ankle pain', 'annual', 'another', 'another round', 'answer', 'answered', 'anxiety', 'anything', 'ape', 'ape lab', 'ape labs', 'apnea', 'appointment', 'appointment doctor', 'appointment needed', 'appointment req', 'appointment requested', 'appointment tomorrow', 'approved', 'appt', 'appt made', 'appt needed', 'appt req', 'area', 'aricept', 'arm', 'arm pain', 'ask', 'ask dj', 'ask doctor', 'asking', 'ativan', 'attempt', 'audiology', 'auth', 'authorization', 'back', 'back hip', 'back pain', 'back reg', 'bad', 'bed', 'behavior', 'behind', 'behind ear', 'better', 'biotech', 'bladder', 'blanket', 'bleeding', 'blister', 'blisters', 'bloatin

In [246]:
df_data = pd.DataFrame(dense_data)
df_summary = pd.DataFrame(dense_summary)
print(df_data.shape)
print(df_summary.shape)

(86554, 3000)
(86554, 1000)


In [247]:
df_data.columns = tfidf_data_vectorizer.get_feature_names()
df_summary.columns = tfidf_summary_vectorizer.get_feature_names()

In [248]:
preprocessed_data=pd.concat([df_data,df_summary],axis=1)

In [249]:
df_test_data = pd.DataFrame(dense_test_data)
df_test_summary = pd.DataFrame(dense_test_summary)
print(df_test_data.shape)
print(df_test_summary.shape)

(2279, 3000)
(2279, 1000)


In [250]:
df_test_data.columns=tfidf_data_vectorizer.get_feature_names()
df_test_summary.columns=tfidf_summary_vectorizer.get_feature_names()

In [251]:
df_test_data

,ab,abd,abdomen,abdominal,able,able get,abnormal,abx,accu,accu chek,...,zithromax oral,zithromax pak,zofran,zolpidem,zolpidem tartrate,zoster,zoster vac,zpack,zpak,zyrtec
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.084246
7,0.0,0.0,0.0,0.0,0.154418,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.321490,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [252]:
preprocessed_test_data=pd.concat([df_test_data,df_test_summary],axis=1)
preprocessed_test_data.shape

(2279, 4000)

In [253]:
preprocessed_data=pd.concat([df_data,df_summary],axis=1)
print(preprocessed_data.shape)

(86554, 4000)


In [170]:
#preprocessed_data.columns

In [254]:
train_y=actual_data1.categories
Test_Y=test_data1.categories

In [255]:
X_train, X_test_val, Y_train, Y_test_val = train_test_split(preprocessed_data, train_y, test_size=0.20,random_state=42)

In [257]:
X_train.shape

(69243, 4000)

In [258]:
Counter(Y_train)

Counter({'APPOINTMENTS': 15594,
         'ASK_A_DOCTOR': 25090,
         'LAB': 2713,
         'MISCELLANEOUS': 8650,
         'PRESCRIPTION': 17196})

In [259]:
#random oversampling of minority class
us=RandomUnderSampler(ratio=.20, random_state=42,replacement=True)
X_train_res,Y_train_res = us.fit_sample(X_train, Y_train)

In [260]:
Counter(Y_train_res)

Counter({'APPOINTMENTS': 13565,
         'ASK_A_DOCTOR': 13565,
         'LAB': 2713,
         'MISCELLANEOUS': 13565,
         'PRESCRIPTION': 13565})

In [261]:
import time
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [262]:
start = time.time()
model_random = RandomForestClassifier(n_estimators = 1000,n_jobs = -1,random_state =1,max_features = "auto",max_depth=20 ,min_samples_leaf = 100)
model_fit = model_random.fit(X_train_res,Y_train_res)
print("Training took {:0.3f} seconds".format(time.time()-start))

Training took 419.998 seconds


In [263]:
pred_train = model_fit.predict(X_train_res)
pred_val = model_fit.predict(X_test_val)
pred_test_rf1 = model_fit.predict(preprocessed_test_data)

In [264]:
print('Train_data:',accuracy_score(Y_train_res,pred_train))
print('valid_data:',accuracy_score(Y_test_val,pred_val))
print('Test_data :',accuracy_score(Test_Y,pred_test_rf1))

Train_data: 0.7077738577922875
valid_data: 0.660216047599792
Test_data : 0.2523036419482229


In [267]:
print('\n Train_data: \n',classification_report(Y_train_res,pred_train))
print('\n valid_data: \n',classification_report(Y_test_val,pred_val))
print('\n Test_data : \n',classification_report(Test_Y,pred_test_rf1))


 Train_data: 
                precision    recall  f1-score   support

 APPOINTMENTS       0.73      0.87      0.79     13565
 ASK_A_DOCTOR       0.68      0.50      0.57     13565
          LAB       0.97      0.40      0.56      2713
MISCELLANEOUS       0.64      0.81      0.72     13565
 PRESCRIPTION       0.77      0.71      0.74     13565

  avg / total       0.72      0.71      0.70     56973


 valid_data: 
                precision    recall  f1-score   support

 APPOINTMENTS       0.69      0.87      0.77      3883
 ASK_A_DOCTOR       0.76      0.47      0.58      6247
          LAB       0.97      0.44      0.60       689
MISCELLANEOUS       0.44      0.82      0.58      2198
 PRESCRIPTION       0.72      0.70      0.71      4294

  avg / total       0.70      0.66      0.66     17311


 Test_data : 
                precision    recall  f1-score   support

 APPOINTMENTS       0.36      0.29      0.32       416
 ASK_A_DOCTOR       0.52      0.14      0.22       973
          

In [196]:
print(' Train_data:\n\n',confusion_matrix(Y_train_res,pred_train))
print('\n validataion_data:\n\n',confusion_matrix(Y_test_val,pred_val))
print('\n Test_data:\n\n',confusion_matrix(Test_Y,pred_test_rf1))

 Train_data:

 [[11778   449     7  1179   152]
 [ 2325  6904    18  2061  2257]
 [  275   262  1130   951    95]
 [ 1295   697    22 11038   513]
 [  335  1753     3  1662  9812]]

 validataion_data:

 [[3392  154    3  297   37]
 [1082 3029   12 1062 1062]
 [  57   55  316  237   24]
 [ 227   99    1 1798   73]
 [ 103  585    1  589 3016]]

 Test_data:

 [[ 74  45   0 238  59]
 [108 183   0 538 144]
 [  5   4   0  60   9]
 [ 24  21   1 228  33]
 [ 35  36   0 364  70]]
